In [0]:
# needed to import excel files
%pip install -i https://pypi.org/simple openpyxl

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from datetime import datetime, timedelta 
from getpass import getuser as get_user 
from getpass import getpass as get_password 
from os import environ as enviroment_vars 
from os import setpgrp as set_process_group 
from random import random 
from shlex import split as cmd_split 
from string import punctuation 
from subprocess import Popen, PIPE, STDOUT 

In [0]:
from pyspark import SparkContext, HiveContext, SparkConf  
from pyspark.sql import SparkSession 

ss = SparkSession(sc) 
ss 

In [0]:
spark.conf.set("spark.databricks.io.cache.enabled", "true")

In [0]:
#Calculate initiation turnaround times First Risk to First Response for current YTD
#Create datetime filters for the monthly calculation; NOTE: CHANGE MONTH END DATE
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import current_date
today = datetime.today()

#a = {'CURRENT_YTD_BEG':datetime(today.year, 1, 1), 
 #    'CURRENT_YTD_END':datetime(today.year, today.month - 1, 31),
  #   'PRIOR_YTD_BEG':datetime(today.year - 1, 1, 1),
   #  'PRIOR_YTD_END':datetime(today.year - 1, today.month - 1, 31)}

#ONLY USE BELOW FOR JAN EACH YEAR   
#a = {'CURRENT_YTD_BEG':datetime(today.year-1, 1, 1), 
 #    'CURRENT_YTD_END':datetime(today.year-1, 12, 31),
  #   'PRIOR_YTD_END':datetime(today.year-2, 12, 31),
   #  'PRIOR_YTD_BEG':datetime(today.year-2, 1, 1)}
    
sql = """
select distinct
t1.cmhc_loan_account_nbr,
t1.rec_month,
tat_type.initiation_turnaround_tcd,
case
when tat.turnaround_duration_cnt is null then 'missing'
when tat.turnaround_duration_cnt<=60 then '1 min or less'
when tat.turnaround_duration_cnt>60 and tat.turnaround_duration_cnt<=3600 
then '1 min to 60 min'
when tat.turnaround_duration_cnt>3600 and tat.turnaround_duration_cnt<=7200 
then '1 to 2 hour'
when tat.turnaround_duration_cnt>7200 and tat.turnaround_duration_cnt<=14400 
then '2 to 4 hour'
when tat.turnaround_duration_cnt>14400 then '4 hours and more'
end as TAT

FROM sb_rmo_ho_reporting.test_lender_dashboard2 t1

inner JOIN cmwdw_insurance.tbdw_tgt_initiation_turnaround_summary_fact tat 
on t1.cmhc_loan_account_nbr = tat.cmhc_loan_account_nbr

inner join cmwdw_insurance.tbdw_tgt_initiation_turnaround_type_dim tat_type 
on tat.initiation_turnaround_type_surrkey = tat_type.initiation_turnaround_type_surrkey

inner join cmwdw_insurance.tbdw_tgt_risk_dim risk 
on tat.end_risk_surrkey = risk.risk_surrkey

WHERE tat.loan_turnaround_error_cnt = 0
and tat_type.initiation_turnaround_tcd = '04'
and risk.risk_result_decision_cde not in ('01', '**')
and t1.loan_purpose_type_desc_en = 'PURCHASE'
"""

dfTAT_r =ss.sql(sql)
#count = dfTAT_r.count()    
#print("Table row count: ", count)
#print("Purchase TAT First Risk to First Response dataset creation completed at: ", str(datetime.now()))

In [0]:
#count = dfTAT_r.count()    
#print("Table row count: ", count)

In [0]:
#Calculate initiation turnaround times First Risk to First Decision for current YTD
sql = """
SELECT distinct
t1.cmhc_loan_account_nbr,
t1.rec_month,
tat_type.initiation_turnaround_tcd,
case
when tat.turnaround_duration_cnt  is null then 'missing'
when tat.turnaround_duration_cnt<=60 then '1 min or less'
when tat.turnaround_duration_cnt>60 and tat.turnaround_duration_cnt<=3600 
then '1 min to 60 min'
when tat.turnaround_duration_cnt>3600 and tat.turnaround_duration_cnt<=7200 
then '1 to 2 hour'
when tat.turnaround_duration_cnt>7200 and tat.turnaround_duration_cnt<=14400 
then '2 to 4 hour'
when tat.turnaround_duration_cnt>14400 then '4 hours and more'
end as TAT

FROM sb_rmo_ho_reporting.test_lender_dashboard2 t1
inner JOIN cmwdw_insurance.tbdw_tgt_initiation_turnaround_summary_fact tat 
on t1.cmhc_loan_account_nbr = tat.cmhc_loan_account_nbr
inner join cmwdw_insurance.tbdw_tgt_initiation_turnaround_type_dim tat_type 
on tat.initiation_turnaround_type_surrkey = tat_type.initiation_turnaround_type_surrkey
inner join cmwdw_insurance.tbdw_tgt_risk_dim risk 
on tat.end_risk_surrkey = risk.risk_surrkey

WHERE
tat.loan_turnaround_error_cnt = 0
and tat_type.initiation_turnaround_tcd in ('05')
"""

dfTAT_d =ss.sql(sql)
count = dfTAT_d.count()    
print("Table row count: ", count)
print("Purchase TAT First Risk to First Decision dataset creation completed at: ", str(datetime.now()))

Table row count:  405672
Purchase TAT First Risk to First Decision dataset creation completed at:  2024-09-06 15:37:12.352593


In [0]:
#Calculate initiation turnaround times First Risk to First Decision with R3 for current YTD
sql = """
SELECT distinct
t1.cmhc_loan_account_nbr,
t1.rec_month,
tat_type.initiation_turnaround_tcd,
case
when tat.turnaround_duration_cnt is null then 'missing'
when tat.turnaround_duration_cnt<=60 then '1 min or less'
when tat.turnaround_duration_cnt>60 and tat.turnaround_duration_cnt<=3600 
then '1 min to 60 min'
when tat.turnaround_duration_cnt>3600 and tat.turnaround_duration_cnt<=7200 
then '1 to 2 hour'
when tat.turnaround_duration_cnt>7200 and tat.turnaround_duration_cnt<=14400 
then '2 to 4 hour'
when tat.turnaround_duration_cnt>14400 then '4 hours and more'
end as TAT

FROM sb_rmo_ho_reporting.test_lender_dashboard2 t1
inner JOIN cmwdw_insurance.tbdw_tgt_initiation_turnaround_summary_fact tat 
on t1.cmhc_loan_account_nbr = tat.cmhc_loan_account_nbr
inner join cmwdw_insurance.tbdw_tgt_initiation_turnaround_type_dim tat_type 
on tat.initiation_turnaround_type_surrkey = tat_type.initiation_turnaround_type_surrkey
inner join cmwdw_insurance.tbdw_tgt_risk_dim risk 
on tat.end_risk_surrkey = risk.risk_surrkey
WHERE
tat.loan_turnaround_error_cnt = 0
and tat_type.initiation_turnaround_tcd in ('08')

"""

dfTAT_R3 =ss.sql(sql)
count = dfTAT_R3.count()    
#dfTAT_R3.write.mode("overwrite").parquet("hdfs:///user/mpei/test_TAT_decisionR3")
print("Table row count: ", count)
print("Purchase TAT First Risk to First Response with R3 dataset creation completed at: ", str(datetime.now()))

Table row count:  414779
Purchase TAT First Risk to First Response with R3 dataset creation completed at:  2024-09-06 15:37:15.006071


In [0]:
#Join the 3 TAT dataframes together
dfTAT_R3.registerTempTable("dfTAT_R3")
dfTAT_r.registerTempTable("dfTAT_r")
dfTAT_d.registerTempTable("dfTAT_d")

sqltat = """
select DISTINCT
CASE WHEN DEC.rec_month IS NULL THEN RES.rec_month ELSE DEC.rec_month 
END AS MONTH,
CASE WHEN DEC.cmhc_loan_account_nbr IS NULL THEN RES.cmhc_loan_account_nbr ELSE DEC.cmhc_loan_account_nbr 
END AS cmhc_loan_account_nbr,
CASE WHEN RES.initiation_turnaround_tcd= '04' AND RES.TAT in ('1 min or less','1 min to 60 min','1 to 2 hour') 
then 1 else 0
END AS RESPONSE, 
CASE WHEN RES.initiation_turnaround_tcd= '04' THEN 1 ELSE 0
END AS RESPONSE_TOTAL,
CASE WHEN DEC1.initiation_turnaround_tcd= '05' AND DEC1.TAT in ('1 min or less','1 min to 60 min','1 to 2 hour','2 to 4 hour') 
then 1 ELSE 0
END AS DECISION,
CASE WHEN DEC1.initiation_turnaround_tcd= '05' THEN 1 ELSE 0
END AS DECISION_TOTAL,
CASE WHEN DEC.initiation_turnaround_tcd = '08' AND DEC.TAT in ('1 min or less','1 min to 60 min','1 to 2 hour','2 to 4 hour') 
then 1 ELSE 0
END AS DECISION_R3,
CASE WHEN DEC.initiation_turnaround_tcd = '08' then 1 ELSE 0
END AS DECISION_R3_TOTAL

from dfTAT_R3 DEC
left JOIN dfTAT_d DEC1 
ON DEC1.cmhc_loan_account_nbr=DEC.cmhc_loan_account_nbr
FULL JOIN dfTAT_r res 
ON RES.cmhc_loan_account_nbr=DEC.cmhc_loan_account_nbr
"""

dfTAT = ss.sql(sqltat)
print("All TAT row count: ", dfTAT.count())
print("All TAT dataset creation completed at: ", str(datetime.now()))

/databricks/spark/python/pyspark/sql/dataframe.py:334: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


All TAT row count:  421347
All TAT dataset creation completed at:  2024-09-06 15:37:22.429516


In [0]:
#Add Desjardins Rep File Names
#Join with Main Base Data, First Transmission Referred and Ever Referred Information, and add Best Credit Score from Data Lake
#Calculate Branch Region and Market using data in Data Lake 
#Calculate auto-approvals, ever-referred, referred at first risking
#Add Risk Decision at first risking and Scorecard variables
#Dec 2021: Join to DL to filter for homeowner loan only
import pandas as pd 

Rep_Mobile = pd.read_excel('Rep Mobile.xlsx')
Plan_Transit = pd.read_excel('Plan Transit (2020).xlsx')

Rep_Mobile = Rep_Mobile.astype(str)
Plan_Transit = Plan_Transit.astype(str)

df1 = ss.createDataFrame(Rep_Mobile)
df1.registerTempTable("Rep_Mobile")
df2 = ss.createDataFrame(Plan_Transit)
df2.registerTempTable("Plan_Transit")
dfTAT.registerTempTable("dfTAT")

sqlall = """
select distinct 
t1.*,
las.best_credit_score_last_risking_cnt as best_credit_score,

TAT.RESPONSE,
TAT.RESPONSE_TOTAL,
TAT.DECISION,
TAT.DECISION_TOTAL,
TAT.DECISION_R3,
TAT.DECISION_R3_TOTAL,
TAT.MONTH,
CASE WHEN T1.parent_level1_lender_cde = "49020" THEN mobile.DDH_NIVEAU1 END AS DR,
CASE WHEN T1.parent_level1_lender_cde = "49020" THEN mobile.DDH_NIVEAU2 END AS CDA,

CASE WHEN T1.parent_level1_lender_cde = "49020" THEN tran.PVP END AS PVP,
CASE WHEN T1.parent_level1_lender_cde = "49020" THEN tran.VPR END AS VPR,

'' AS REPORT_DATE,

case when rdt.risk_result_decision_desc_en not in ("AUTO APPROVED") then "MODEL OR AUTO REFERRED" 
else rdt.risk_result_decision_desc_en 
end as CLEAN_RISK_DECISION,

rdt.risk_property_rate_desc_en as PRAMSYS_first,
lad.application_scorecard_first_risking_rating_desc_en as appl_scorecard_first_risking_rating,
lts.application_scorecard_total_pts as appl_scorecard_total_pts_first,
lts.borrower_model_rating_pts as borrower_model_rating_pts_first,
lts.application_scorecard_borrower_total_pts as borr_sccard_total_pts_first,

case
when trim(rdt.risk_result_decision_cde) in ('01') then 'Auto-Approved'
when trim(rdt.risk_result_decision_cde) in ('06') then 'RU-Approved'
when trim(rdt.risk_result_decision_cde) in ('07') then 'RU-Declined'
when trim(rdt.risk_result_decision_cde) in ('08') then 'RU-Reffered'
when trim(rdt.risk_result_decision_cde) in ('02') then 'Auto-Declined'
when trim(rdt.risk_result_decision_cde) in ('12','13','14','15','16','17','18','19','20','21','22','23','24','25') 
then 'Auto Decline Incremental'
when trim(rdt.risk_result_decision_cde) in ('03')  then 'Model Referred'
when trim(rdt.risk_result_decision_cde) in ('09')  then 'RU-Referred by models only'
when trim(rdt.risk_result_decision_cde) in ('04')  then 'Policy Referred'
when trim(rdt.risk_result_decision_cde) in ('10')  then 'RU - Policy Referred'
when trim(rdt.risk_result_decision_cde) in ('05')  then 'Model and Policy Referred'
when trim(rdt.risk_result_decision_cde) in ('11')  then 'RU-Model and Policy Referred'
else 'Other'
end as Risk_decision_first,
rdt.risk_result_decision_cde,

lad.underwriting_ever_referred_cde,
CASE 
    WHEN trim(t2.civic_address_province_nm_en) IN 
    ('NEWFOUNDLAND AND LABRADOR', 'NEW BRUNSWICK', 'PRINCE EDWARD ISLAND', 'NOVA SCOTIA') THEN 'ATLANTIC' 
    WHEN trim(t2.civic_address_province_nm_en) IN 
    ('ALBERTA', 'MANITOBA', 'SASKATCHEWAN', 'YUKON', 'NUNAVUT', 'NORTHWEST TERRITORIES')
    THEN 'PRAIRIES'
    ELSE t2.civic_address_province_nm_en 
    END AS BRANCH_REGION,

CASE 
WHEN t2.civic_address_province_nm_en IN ('ALBERTA', 'MANITOBA', 'SASKATCHEWAN', 'YUKON', 'NUNAVUT', 'NORTHWEST TERRITORIES')
and concat(trim(t1.current_stats_canada_division_cde), trim(t1.current_stats_canada_subdivision_cde)) in ('06016') 
THEN 'Calgary' 
WHEN t2.civic_address_province_nm_en IN ('ALBERTA', 'MANITOBA', 'SASKATCHEWAN', 'YUKON', 'NUNAVUT', 'NORTHWEST TERRITORIES')
and concat(trim(t1.current_stats_canada_division_cde), trim(t1.current_stats_canada_subdivision_cde)) in ('11061') 
THEN 'Edmonton' 
WHEN t2.civic_address_province_nm_en IN ('BRITISH COLUMBIA') 
and concat(trim(t1.current_stats_canada_division_cde), trim(t1.current_stats_canada_subdivision_cde)) 
IN ( '15022', '15020', '15046', '15051', '15055' ) 
THEN 'Vancouver' 
WHEN t2.civic_address_province_nm_en IN ('ONTARIO') 
and concat(trim(t1.current_stats_canada_division_cde), trim(t1.current_stats_canada_subdivision_cde)) 
IN ('18005', '19046', '21010', '18039', '24002', '21024', '18017', '19054', '20019', '19070', '24015', '19049', '19036', '24009', '21005', '19048', '20008', '24001', '18013', '19038', '18001', '20001', '18020', '20004', '20005', '18029', '19028', '18009', '19044', '20014' ) 
THEN 'GTA' 
WHEN t2.civic_address_province_nm_en IN ('QUEBEC')
and concat(trim(t1.current_stats_canada_division_cde), trim(t1.current_stats_canada_subdivision_cde)) IN ( '66023', '66007', '66047' ) 
THEN 'Montreal' 
ELSE 'Other' 
END AS Mkt,

concat(trim(t1.transit),' - ', trim(t2.civic_address_street_nbr), ' - ', trim(t2.civic_address_street_nm), ', ', trim(t2.civic_address_municipality_nm)) as TRANSIT_ID

from sb_rmo_ho_reporting.test_lender_dashboard2 t1

left join dfTAT TAT
on t1.cmhc_loan_account_nbr = TAT.cmhc_loan_account_nbr

left join Rep_Mobile mobile
on mobile.NOM_DDH =t1.Original_Rep

left join Plan_Transit tran
on tran.Transit=t1.Transit_DESJ

left join cmwdw_insurance.vwdw_loan_account_dim lad
on t1.cmhc_loan_account_nbr = lad.cmhc_loan_account_nbr

left join cmwdw_insurance.vwdw_loan_account_sum las
on las.loan_account_surrkey = lad.loan_account_surrkey

left join cmwdw_insurance.vwdw_loan_transmission_dim ltd 
on (t1.cmhc_loan_account_nbr = ltd.cmhc_loan_account_nbr and ltd.risked_first_ind = 1) 

left join cmwdw_insurance.tbdw_tgt_risk_dim rdt 
on (ltd.cmhc_loan_account_nbr = rdt.cmhc_loan_account_nbr and ltd.application_identifier_ts = rdt.application_identifier_ts)

left join cmwdw_insurance.vwdw_loan_transmission_sum lts
on lts.loan_transmission_surrkey = ltd.loan_transmission_surrkey

left join cmwdw_insurance.vwdw_lender_financial_trnst_dim t2
on t2.lender_financial_transit_surrkey = las.initiation_lender_financial_transit_surrkey

left join cmwdw_insurance.vwdw_costable_unit_dim cu
on cu.costable_unit_surrkey = las.costable_unit_surrkey

where cu.insurance_product_group_cde = '100'
 """

#dfall = pd.read_sql(sqlall.format(a.get('CURRENT_YTD_BEG'), a.get('CURRENT_YTD_END')))
#dfall = dfall.dropna(axis='columns', how='all') 
spark_df = ss.sql(sqlall)
spark_df = spark_df.withColumn("Rundate",current_date())
spark_df.registerTempTable("spark_df")
#spark_df.write.mode("overwrite").format("orc").saveAsTable("sb_rmo_ho_reporting.lender_dashboard_tat_desj")
#print("Final table row count: ", spark_df.count())
print("Table creation for Main Data with TAT and Desjardins Rep completed at: ", str(datetime.now()))

/databricks/spark/python/pyspark/sql/dataframe.py:334: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)
/databricks/spark/python/pyspark/sql/dataframe.py:334: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


Table creation for Main Data with TAT and Desjardins Rep completed at:  2024-09-06 15:37:26.629036


In [0]:
#distinct_count = spark_df.select("parent_level1_lender_nm_en").distinct().count()
#print("Distinct count:", distinct_count)

In [0]:
#Create borrower scorecard rating result DF, at first risking
sqlborr = """
select distinct
t1.cmhc_loan_account_nbr, 
t1.scorecard_model_rating_type_desc_en as borrower_scorecard_rating

from cmwdw_insurance.vwdw_borrower_transm_basic_dim t1
inner join 
(select cmhc_loan_account_nbr, min(application_identifier_ts) as first_risking

from cmwdw_insurance.vwdw_borrower_transm_basic_dim 

where borrower_sequence_id = 1 and application_identifier_ts >= '2016-01-01' and application_identifier_ts < '9999-12-01'
and scorecard_model_rating_tcd not in ('**')
group by cmhc_loan_account_nbr) as t2
on t1.cmhc_loan_account_nbr = t2.cmhc_loan_account_nbr and t1.application_identifier_ts = t2.first_risking
where t1.borrower_sequence_id = 1 
and t1.application_identifier_ts >= '2016-01-01' and t1.application_identifier_ts < '9999-12-01'
"""
df_borr = ss.sql(sqlborr)
df_borr.registerTempTable("df_borr")

#Create Co-borrower scorecard rating result DF, at first risking
sqlcoborr = """
select distinct 
t1.cmhc_loan_account_nbr, 
t1.scorecard_model_rating_type_desc_en as coborrower_scorecard_rating

from cmwdw_insurance.vwdw_borrower_transm_basic_dim t1
inner join 
(select cmhc_loan_account_nbr, min(application_identifier_ts) as first_risking
    from cmwdw_insurance.vwdw_borrower_transm_basic_dim 

where borrower_sequence_id = 2 and application_identifier_ts >= '2016-01-01' and application_identifier_ts < '9999-12-01'
and scorecard_model_rating_tcd not in ('**')

group by cmhc_loan_account_nbr) as t2
on t1.cmhc_loan_account_nbr = t2.cmhc_loan_account_nbr and t1.application_identifier_ts = t2.first_risking
where t1.borrower_sequence_id = 2 
and t1.application_identifier_ts >= '2016-01-01' and t1.application_identifier_ts < '9999-12-01'
"""
df_coborr = ss.sql(sqlcoborr)
df_coborr.registerTempTable("df_coborr")

/databricks/spark/python/pyspark/sql/dataframe.py:334: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [0]:
#Merge with main table and save in Data Lake
sqlallborr = """
select distinct 
t1.*,
t2.borrower_scorecard_rating,
t3.coborrower_scorecard_rating
from spark_df t1
left join df_borr t2 
on t1.cmhc_loan_account_nbr = t2.cmhc_loan_account_nbr
left join df_coborr t3 
on t1.cmhc_loan_account_nbr = t3.cmhc_loan_account_nbr
"""
df_allborr = ss.sql(sqlallborr)
df_allborr.write.mode("overwrite").format("orc").saveAsTable("sb_rmo_ho_reporting.lender_dashboard_tat_desj")
print("Final table with Scorecard results row count: ", df_allborr.count())
print("Table creation for Main Data with TAT, Desjardins Rep, Scorecard results completed at: ", str(datetime.now()))

Final table with Scorecard results row count:  430286
Table creation for Main Data with TAT, Desjardins Rep, Scorecard results completed at:  2024-09-06 15:38:31.532760


In [0]:
#distinct_count = df_allborr.select("PARENT_LENDER_EN").distinct().count()
#print("Distinct count:", distinct_count)

#distinct_count = df_allborr.select("PARENT_LENDER_EN").distinct()
#distinct_count.display()

In [0]:
#Summarize TAT data for Power BI Dashboard
sqlsum = """
select distinct
TRANSIT_ID,
LENDER_EN,
LENDER_CDE,
PARENT_LENDER_EN, 
PARENT_LENDER_CODE,
MONTH,
SOURCE,
CMHC_SOURCE,
MOSE_CHANNEL,
LENDER_GROUP,
REGION,
BRANCH_REGION,
Mkt,
BRM_RVP,
HFLM_RD,
SALES_MANAGER,
HFS_RD,
DR,
CDA,
PVP,
VPR,
COUNT(RESPONSE)AS RESPONSE_CNT,
COUNT(RESPONSE_TOTAL)AS RESPONSE_TOTAL_CNT,
COUNT(DECISION) AS DECISION_CNT,
COUNT(DECISION_TOTAL) AS DECISION_TOTAL_CNT,
COUNT(DECISION_R3) AS DECISIONR3_CNT,
COUNT(DECISION_R3_TOTAL) AS DECISIONR3_TOTAL_CNT

from sb_rmo_ho_reporting.lender_dashboard_tat_desj
group by 
TRANSIT_ID,
LENDER_EN, 
LENDER_CDE,
PARENT_LENDER_EN, 
PARENT_LENDER_CODE,
MONTH,
SOURCE,
CMHC_SOURCE,
MOSE_CHANNEL,
LENDER_GROUP,
REGION,
BRANCH_REGION,
Mkt,
BRM_RVP,
HFLM_RD,
SALES_MANAGER,
HFS_RD,
DR,
CDA,
PVP,
VPR
"""

df_sum = ss.sql(sqlsum)
df_sum.write.mode("overwrite").format("orc").saveAsTable("sb_rmo_ho_reporting.lender_dashboard_summary_tat")
print("Summary TAT table row count: ", df_sum.count())
print("Table creation for Summary TAT completed at: ", str(datetime.now()))

Summary TAT table row count:  66846
Table creation for Summary TAT completed at:  2024-09-06 15:38:37.308964


In [0]:
#distinct_count = df_sum.select("PARENT_LENDER_EN").distinct().count()
#print("Distinct count:", distinct_count)